In [1]:
# load and evaluate a saved model
from numpy import loadtxt
from keras.models import load_model
import pickle
import pandas as pd 
import numpy as np
import time 

Using TensorFlow backend.


In [2]:
#load model
model = load_model("./models/dae_cnn.h5")
# summarize model.
model.summary()
with open("./models/LabelEncoders_dic.pickle","rb") as f:
    encoder_dic=pickle.load(f)
with open("./models/MinMaxScalers_dic.pickle","rb") as f:
    scaler_dic=pickle.load(f)

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 115, 1)       0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 115, 115)     345         input_2[0][0]                    
__________________________________________________________________________________________________
max_pooling1d_1 (MaxPooling1D)  (None, 57, 115)      0           conv1d_1[0][0]                   
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 57, 57)       13167       max_pooling1d_1[0][0]            
____________________________________________________________________________________________

In [3]:
def transformer_df(df):
    for c in df.columns :
        if (df[c].dtype =="object"):
            encoder = encoder_dic[c]
            df[c]=encoder.transform(df[c])
    return df

In [4]:
def normaliser_all_columns(df):
    for c in df.columns :
        df[c]=scaler_dic[c].transform(df[c].values.reshape(-1,1))
    return df

In [11]:
def reshape_x_data_3d(xtrain):
    return np.array(xtrain).reshape(xtrain.shape[0],xtrain.shape[1],1)

In [5]:
df =pd.read_csv("../../../data/full_data_small_datased.csv")

In [6]:
test_df = pd.DataFrame()
test_df = test_df.append(df.iloc[0])
test_df.head()

,Class,HH_L0.01_covariance,HH_L0.01_magnitude,HH_L0.01_mean,HH_L0.01_pcc,HH_L0.01_radius,HH_L0.01_std,HH_L0.01_weight,HH_L0.1_covariance,HH_L0.1_magnitude,...,MI_dir_L0.1_weight,MI_dir_L1_mean,MI_dir_L1_variance,MI_dir_L1_weight,MI_dir_L3_mean,MI_dir_L3_variance,MI_dir_L3_weight,MI_dir_L5_mean,MI_dir_L5_variance,MI_dir_L5_weight
0,0.0,0.0,60.0,60.0,0.0,0.0,0.0,1.0,0.0,60.0,...,1.0,60.0,0.0,1.0,60.0,0.0,1.0,60.0,0.0,1.0


In [7]:
target = test_df.Class

In [8]:
x_test = test_df.drop(['Class'],axis=1)

In [9]:
x_test=transformer_df(x_test)
x_test=normaliser_all_columns(x_test)

In [12]:
y_pred=np.round(model.predict([x_test,reshape_x_data_3d(x_test)])) 

In [14]:
y_pred[0]

array([0.], dtype=float32)